如果一个网络完全由线性层串联起来，就叫全连接(fully connected)的网络，例如多分类问题中采用的网络，![](https://pic.imgdb.cn/item/659ce3a7871b83018a4b96c5.jpg)  

而经典的卷积神经网络的流程如下图所示：  
![](https://pic.imgdb.cn/item/659d0e4b871b83018acb2773.jpg)  
一般分为两个阶段：特征提取（feature extraction）以及分类（classification）：  
特征提取：  
卷积（convolution）和下采样（subsampling），其中下采样中多采用最大池化（maxpooling）  
分类：  
将最终提取到的特征，进行一次全连接，再通过维度变换成最终结果的维度。


## 卷积层
通常情况下，我们处理的图像是栅格图像，即图像由一个个的像素点组成，每一个像素点又有RGB三个通道值共同组成。
通常我们采用 C * W * H 表示整个图像的信息，即channel * width * height  
而在卷积层中，输入输出是四维张量：[batch, channel, width, height]，第一分量即样本数。`transforms.ToTensor`转化后的图像就是这种的四维张量，因此不需要额外排列。  
![](https://pic.imgdb.cn/item/659d10c0871b83018ad20dad.jpg) 
### 卷积核 
卷积层所做的操作实际上很简单，就是对应元素相乘相加，对于卷积核（convolution kernel）同大小的输入，完成计算，并输出到对应位置即可。而每一个通道都对应一个卷积核，每个通道计算的结果再相加，为这一次卷积最终结果。
> tips：对于 $m * m$ 大小的卷积核，考虑其需要从 $m / 2$的位置开始和结束计算，因此，对于输入的维度减少为：$2 * (m / 2)$ ，这里用的是整除，例如，对于[1, 28, 28]的输入，采用卷积核为$5 * 5$，则输出的维度为[1, 24, 24]  

### 通道数
卷积层实际上还可以更改通道数，若最终结果的通道数为m，则需要m个filter。
![](https://pic.imgdb.cn/item/659d141a871b83018adb886b.jpg)  
### padding
如果想要卷积前后的维度不变，则需要用到padding，即往外延伸，如下图所示为$padding=1$   
![](https://pic.imgdb.cn/item/659d1557871b83018adf1d10.jpg)  
### stride
stride为每一次行走的步长，下图为$stride=2$的示意：  
![](https://pic.imgdb.cn/item/659d15e5871b83018ae0b9cf.jpg)  

## 下采样层
这里采用最大池化，如果最大池的核数为2，则默认情况下起stride也为2  
![](https://pic.imgdb.cn/item/659d161a871b83018ae15a1f.jpg)   


整个特征提取的过程为：  
![](https://pic.imgdb.cn/item/659d1692871b83018ae2bef1.jpg)  
可以发现，在卷积层和最大池层，并不要求知道样本的维度具体值，只有在最后的全连接层，需要知道最终得到了多少个像素，这一步需要计算，也可以先用一个简单样本输入模型，输出经过最大池化的维度来判断

In [10]:
import torch
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import numpy as np

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307, ), (0.3081, ))
])

train_data = datasets.MNIST(root='./dataset/mnist/',
                            train=True,
                            download=False,
                            transform=transform)
train_dataloader = DataLoader(dataset=train_data,
                              batch_size=64,
                              shuffle=True)

test_data = datasets.MNIST(root='./dataset/mnist/',
                           train=False,
                           download=False,
                           transform=transform)
test_dataloader = DataLoader(dataset=test_data,
                             batch_size=64,
                             shuffle=False)

In [11]:
import torch.nn.functional as F
class Net(torch.nn.Module):
    def __init__(self) -> None:
        super(Net,self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = torch.nn.Conv2d(10, 20, kernel_size=5)
        self.maxpooling = torch.nn.MaxPool2d(kernel_size=2)
        self.fc = torch.nn.Linear(320, 10)
    
    def forward(self, x):
        batch = x.size(0)
        x = self.maxpooling(F.relu(self.conv1(x)))
        x = self.maxpooling(F.relu(self.conv2(x)))
        x = x.view(batch, -1)               # 每一行就是一个样本
        return self.fc(x)

model = Net()
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")   如果有GPU
# model.to(device)

In [12]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.01, momentum=0.5)

In [13]:
def train():
    for idx, data in enumerate(train_dataloader, 0):
        inputs, labels = data
        # inputs, labels = inputs.to(device), labels.to(device) 如果有GPU
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_dataloader:
            inputs, labels = data
            outputs = model(inputs)
            _, pred = torch.max(outputs.data, dim=1)
            correct += (pred == labels).sum().item()
            total += labels.size(0)
    print("acc is %lf" % (correct / total))

In [14]:
if __name__ == "__main__":
    for epoch in range(100):
        train()
        if epoch % 10 == 9:
            test()

acc is 0.987400
acc is 0.989900
acc is 0.988300
acc is 0.990200
acc is 0.989100
acc is 0.989800
acc is 0.989000
acc is 0.989900
acc is 0.989200
acc is 0.989900
